<img src="uva_seal.png">  

## Sparsity

### University of Virginia
### DS 5110: Big Data Systems
### Last Updated: February 15, 2023

---  

### SOURCES 

[Spark SparseMatrix documentation](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.mllib.linalg.SparseMatrix.html)  
[Compressed Column Storage format example](https://netlib.org/linalg/html_templates/node92.html#SECTION00931200000000000000)

### OBJECTIVES
1. Understand the benefits of sparsity
2. Calculate the dot product of two sparse vectors
3. Understand how CSC format works


### CONCEPTS AND FUNCTIONS
- SparseVector
- SparseMatrix
- CSC format

---  

### 1. Introduction to Sparsity

Imagine a vector $v$ of length 1 million where one value is 10 and the rest are zero.  
Another vector $w$ is also length 1 million. A value in a different position is 4, and the rest of the values are zero.  
We want to store $v$ and $w$.  
We also want to compute their dot product:

$ v \cdot w = \sum_i v_i w_i$

**Storage and compute in the usual way are extremely inefficient.**

How can we be more efficient? Take advantage of the zeros, or *sparsity.*

A *sparse* object contains mostly zeros. The opposite is *dense*.

Sparse vectors and matrices come up a lot in ML. 

Examples:

- An interaction matrix containing Amazon customers on rows, products on columns, 
and elements representing the number of times each user purchased each item.

- A term-document matrix containing documents on rows, words in a vocabulary on columns,
and elements representing the number of times each word appears in each document.

In each case, most elements will be zero.

Spark takes advantage of sparsity whenever possible.

Sparse vectors and matrices assume that elements have value zero unless otherwise stated.

### 2. Saving on Storage and Compute

**Storage**  
If we store only what's nonzero, the required storage requirement can be much lower.  

Here is an example of a Sparse Vector in Spark:

```
SparseVector(100000, {0: 1.0, 1162: 3.0})
```

It stores three pieces of information:

- vector length (100000 in this case)
- positions of non-zero elements (0 and 1162)
- values of non-zero elements (1 and 3)

**The required storage is much lower for this sparse vector versus the dense vector**

Also note that non-zero positions and values must be associated in some way.  

Here they are saved as key : value pairs.  
- position 0 holds value 1. position 1162 holds value 3.

It is also common to provide two lists with corresponding elements like this:
- [0, 1162]    # locations
- [1.0, 3.0]  # values

which can have form
```
SparseVector(100000, [0, 1162], [1.0, 3.0])
```

**Compute**

We can define and use methods that know how to compute on sparse objects.  

This can reduce the number of operations.

Returning to the dot product, we can

1. create the sparse objects
2. compute a product $v_i w_i$ only when both $v_i$ and $w_i$ are non-zero for some $i$ 


```
v = SparseVector(100000, [0, 5, 1162], [1.0, 3.0, 6.0])
w = SparseVector(100000, [2, 5, 1162], [4.0, 1.0, 2.0])
```

In positions 5 and 1162, both vectors have non-zero elements.  We compute element-wise products like this:

$
v \cdot w = (3 \times 1) + (6 \times 2) = 15
$

If we did this calculation with dense vectors, we would need 1 million multiplications and 999,999 additions.


### 3. Sparse Matrices

There are different ways to store a sparse matrix.

Spark includes the *Sparse Matrix* object. It is stored in Compressed Sparse Column (CSC) format.

It is column oriented 

Benefits include:
- efficient storage
- efficient column slicing
- fast matrix vector products 

Here is an example, let's see if we can figure out how it works.

<img src="matrix.png">  

<img src="ccs.png">  

Let NNZ = number of nonzero elements

Each nonzero element is stored in $val$

The row of each nonzero element is stored in $row\_ind$, where the first row is index 1.

The $col\_ptr$ stores the index of the elements in $val$ which start a column.

**TRY FOR YOURSELF (UNGRADED EXERCISES)**

1) Consider two vectors of length 1 million.  
The first has a nonzero element in the first position.  
The second has a nonzero element in the last position.  
What is their dot product?

Answer: As there is no overlap in nonzero entry positions, their dot product is zero.

2) Think of another method for efficiently storing a sparse matrix.  
What are the benefits and drawbacks compared to the CSC format?

Answer: CSR format for example.